## Homework-4 ##

### Question - 5 

Solve the following SDP problem using the cutting plane algorithm in python. 

$$
\begin{aligned}
    \max_{X\in \mathcal{S}^{4}} \sum_{i,j,i\neq j} Q_{ij} \cdot (1 - X_{ij})\\
\text{diag}(X) = e\\
X\succeq 0 
\end{aligned}
$$

where
$
Q = \begin{bmatrix}
0 & 1 & 3 & 1 \\
1 & 0 & 0 & 2 \\
3 & 0 & 0 & 4 \\
1 &2 & 4 & 0 \\
\end{bmatrix}
$


$\text{diag}(X) = e$ means the diagonal entries of $X$ equal to 1. 

In [1]:
# Including libraries

from pyomo.environ import *
import numpy as np
import pprint

In [2]:
# Creating a Pyomo model

model = ConcreteModel()

# Define sets

model.I = RangeSet(0,3)

# Define variables
model.X = Var(model.I, model.I)

# Define parameters
def Q_init(model, i, j):
    Q = np.array([[0, 1, 3, 1], [1, 0, 0, 2], [3, 0, 0, 4], [1, 2, 4, 0]])
    # Q = np.array([[10, -4, -12, -4], [-4, 6, 0, -8], [-12, 0, 14, -16], [-4, -8, -16, 14]])
    return Q[i, j]

model.Q = Param(model.I, model.I, initialize=Q_init)

model.Q.pprint()

Q : Size=16, Index=Q_index, Domain=Any, Default=None, Mutable=False
    Key    : Value
    (0, 0) :     0
    (0, 1) :     1
    (0, 2) :     3
    (0, 3) :     1
    (1, 0) :     1
    (1, 1) :     0
    (1, 2) :     0
    (1, 3) :     2
    (2, 0) :     3
    (2, 1) :     0
    (2, 2) :     0
    (2, 3) :     4
    (3, 0) :     1
    (3, 1) :     2
    (3, 2) :     4
    (3, 3) :     0


In the first iteration of the cutting plane algorithm, we only have the linear constraint $(X_{ii} = 1), \forall i = (1,3)$. Hence, the problem is unbounded in the first iteration. To make the problem bounded, we can introduce an additional constraint by adding an upper bound on the off-diagonal elements (just for the first iteration) of the $X$. Let's calculate a tight upper bound using out linear algebra knowledge

We know, if a matrix $X$ is PSD, we have:

$$ v^T X v \geq 0 $$



In [3]:
# Define objective function

model.obj = Objective(expr= sum(model.Q[(i,j)]*(1 - model.X[(i,j)]) for i in model.I for j in model.I), sense=maximize)

model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 0*(1 - X[0,0]) + (1 - X[0,1]) + 3*(1 - X[0,2]) + (1 - X[0,3]) + (1 - X[1,0]) + 0*(1 - X[1,1]) + 0*(1 - X[1,2]) + 2*(1 - X[1,3]) + 3*(1 - X[2,0]) + 0*(1 - X[2,1]) + 0*(1 - X[2,2]) + 4*(1 - X[2,3]) + (1 - X[3,0]) + 2*(1 - X[3,1]) + 4*(1 - X[3,2]) + 0*(1 - X[3,3])


In [4]:
# Define constraints

def const_rule1(model, i): # Constraint rule to set the diagonal values of X to be 1
    return model.X[(i,i)] == 1

model.const1 = Constraint(model.I, rule = const_rule1)


def const_rule2(model, i, j):
    if i !=j:
        return model.X[(i,j)] <= 1
    else:
        return Constraint.Skip
    
model.const2 = Constraint(model.I, model.I, rule = const_rule2)

def const_rule3(model, i, j):
    if i !=j:
        return model.X[(i,j)] >= -1
    else:
        return Constraint.Skip
    
model.const3 = Constraint(model.I, model.I, rule = const_rule3)

In [5]:
# Setting the solver
solver = SolverFactory('gurobi')

#  Cutting plane algorithm:

i = 0

while True:
    
    results = solver.solve(model)

    # if i==0:
    #     del model.const2

    # Check for solver termination status
    if results.solver.termination_condition != TerminationCondition.optimal:
        print("Solver failed to converge.")
        print("At iteration", i)
        break
    
    # Extract the optimal solution
    X_val = np.array([[model.X[i, j].value for j in model.I] for i in model.I])
    
    # Perform eigenvalue decomposition
    eigenvalues, eigenvectors = np.linalg.eig(X_val)

    print(eigenvectors)
    
    # Find the smallest eigenvalue
    smallest_eigenvalue = min(eigenvalues)

    # Check termination condition
    if smallest_eigenvalue >= -1e-4:
        print("Optimal solution found.")
        break
    
    # Find the eigen vector corresponding to the smallest eigen value
    v = eigenvectors[:, np.argmin(eigenvalues)]
    
    # Adding additional constraints
    model.const4 = ConstraintList()

    # Add the violated constraint
    model.const4.add(expr= sum(v[k]*model.X[(k,j)]*v[j] for k in model.I for j in model.I) >= 0)

    i+=1
    print(i)

# Print optimal solution
print("Optimal solution:")
for i in range(3):
    for j in range(3):
        print(f"X[{i+1},{j+1}] = {model.X[i, j].value}")

[[ 0.8660254  -0.5         0.3528287  -0.25586923]
 [-0.28867513 -0.5        -0.85253215  0.31153791]
 [-0.28867513 -0.5         0.14065217 -0.67433476]
 [-0.28867513 -0.5         0.35905128  0.61866607]]
1
[[-2.18862163e-01+0.j         -2.69197041e-01-0.49697225j
  -2.69197041e-01+0.49697225j  1.46160482e-16+0.j        ]
 [ 6.06093315e-01+0.j          3.24138714e-01-0.10142937j
   3.24138714e-01+0.10142937j -2.22044605e-16+0.j        ]
 [ 6.06093315e-01+0.j          3.24138714e-01-0.10142937j
   3.24138714e-01+0.10142937j  7.07106781e-01+0.j        ]
 [-4.66262953e-01+0.j          6.70704029e-01+0.j
   6.70704029e-01-0.j          7.07106781e-01+0.j        ]]
Optimal solution found.
Optimal solution:
X[1,1] = 1.0
X[1,2] = 1.0
X[1,3] = -1.0
X[2,1] = -1.0
X[2,2] = 1.0
X[2,3] = 1.0
X[3,1] = -1.0
X[3,2] = 1.0
X[3,3] = 1.0
